In [45]:
import pandas as pd
from vncorenlp import VnCoreNLP
import ast

In [70]:
df = pd.read_csv('./data/data_labeled_updated.csv').drop('Unnamed: 0', axis=1)
df.head()

,NewsTitle,NewsFullContent,label
0,VCB ký hợp đồng vay vốn 100 triệu USD,Ông Nguyễn Hòa Bình Chủ tịch HĐQT của VCB cho ...,"[('ông', 0), ('nguyễn hoà bình chủ tịch', 0), ..."
1,"Kết thúc Phiên 1, 30/10: STB và PPC đang &quot...",Trong khi thị trường Châu Âu có phiên tăng điể...,"[('trong', 0), ('khi', 0), ('thị trường', 1), ..."
2,"Ngày 23/3, Hose đón tân binh PNJ",Kế hoạch này đã được đại diện PNJ bà Cao Thị N...,"[('kế hoạch', 1), ('này', 0), ('đã', 0), ('đượ..."
3,Sắp khởi công Đại học FPT,Theo đó tháng 7/2009 Đại học FPT sẽ chính thức...,"[('theo', 1), ('đó', 0), ('tháng', 1), ('7/200..."
4,SSI đứng đầu thị phần môi giới cổ phiếu,Trong khi chỉ có 5 công ty nắm tới trên 30 ngh...,"[('trong', 0), ('khi', 0), ('chỉ', 1), ('có', ..."


In [71]:
type(df.loc[0, 'label'])

str

# Label

In [72]:
df['list_label'] = df['label'].apply(lambda x: ast.literal_eval(x))

In [75]:
def get_y(text):
    return [word[1] for word in text]

y = df['list_label'].apply(lambda x: get_y(x)).to_list()

# NER feature

In [5]:
annotator = VnCoreNLP(r"./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg", max_heap_size='-Xmx500m') 

In [49]:
def word_segment(text):
    annotated_text = annotator.annotate(text)['sentences'][0]
    return [dic['form'].replace('_',' ').lower() for dic in annotated_text] 

In [50]:
df['word_segment'] = df['NewsFullContent'].apply(lambda text: word_segment(text))

In [51]:
ner = pd.read_csv('./data/ner.csv').drop('Unnamed: 0', axis=1)
ner['ner'] = ner['ner'].apply(lambda x: ast.literal_eval(x))
ner.head()

,ner
0,"[vcb, intesa sanpaolo, italia, cửu long, hà nộ..."
1,"[châu âu, hastc index, nhiệt điện, thương mại,..."
2,"[sfc, nhiên liệu, chứng khoán, thương mại, vàn..."
3,"[fpt, công ty, đại học, nihon sekkei nhật bản,..."
4,"[sài gòn, hose, á châu, ck, bsc, ngoại thương,..."


In [52]:
df = pd.concat([df, ner], axis=1)

In [57]:
def ner_tag(row):
    keywords = row['ner']
    text = row['word_segment']
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']

    text_df['tag'] = text_df['word'].apply(lambda x: tag(x, keywords))
    
#     return list(zip(text_df['word'], text_df['tag']))
    return list(text_df['tag'])

In [58]:
name_entity_reg = []
for i,row in df.iterrows():
    name_entity_reg.append(ner_tag(row))

df['name_entity'] = name_entity_reg

In [83]:
# ner = VnCoreNLP("./VnCoreNLP-master/VnCoreNLP-1.1.1.jar", annotators="wseg,pos,ner", max_heap_size='-Xmx2g')

In [20]:
# def vnner(text):
#     annotated_text = ner.annotate(text)
#     pos = []
#     for dic in annotated_text['sentences'][0]:
#         if dic['nerLabel'] != 'O':
#             pos.append(dic['form'].replace('_',' ').lower())
        
#     return list(set(pos))

# def tag(word, dic):
#     if word in dic:
#         return True
#     else:
#         return False

# def ner_tag(row):
#     keywords = vnner(row['NewsFullContent'])
#     text = row['word_segment']
#     text_df = pd.DataFrame(text)
#     text_df.columns = ['word']

#     text_df['tag'] = text_df['word'].apply(lambda x: tag(x, keywords))
    
# #     return list(zip(text_df['word'], text_df['tag']))
#     return list(text_df['tag'])

# Title

In [61]:
def tag(word, dic):
    if word in dic:
        return True
    else:
        return False

def title_detect(row):
    annotated_text = annotator.annotate(row['NewsTitle'])['sentences'][0]
    title = [dic['form'].replace('_',' ').lower() for dic in annotated_text] 
    
    text = row['word_segment']
    text_df = pd.DataFrame(text)
    text_df.columns = ['word']

    text_df['tag'] = text_df['word'].apply(lambda x: tag(x, title))
    
    return list(text_df['tag'])

In [62]:
title_tag = []
for i,row in df.iterrows():
    title_tag.append(title_detect(row))

df['title_tag'] = title_tag

In [64]:
df[['name_entity', 'title_tag']].to_csv('./data/ner_title_feature.csv')

In [69]:
df[['name_entity', 'title_tag']]

,name_entity,title_tag
0,"[False, False, False, False, True, False, Fals...","[False, False, False, False, True, False, Fals..."
1,"[False, False, False, True, False, False, Fals...","[False, False, False, False, False, True, Fals..."
2,"[False, False, False, False, False, True, Fals...","[False, False, False, False, False, True, Fals..."
3,"[False, False, False, False, True, True, False...","[False, False, False, False, True, True, False..."
4,"[False, False, False, False, False, False, Fal...","[False, False, False, False, False, False, Fal..."
...,...,...
644,"[False, True, True, True, True, False, False, ...","[False, False, False, False, False, False, Fal..."
645,"[False, True, True, True, True, False, False, ...","[False, False, False, False, False, False, Fal..."
646,"[False, False, False, False, False, False, Fal...","[False, True, False, False, False, False, Fals..."
647,"[False, True, True, False, False, False, False...","[False, False, True, False, False, False, Fals..."
